# AirBNB tracker

In [9]:
import pyperclip
import re

In [98]:
temp = pyperclip.paste()

In [100]:
def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    stop = texte.find('1\n2\n3\n4')
    return texte[:stop].strip()

In [101]:
temp = tronque_à_gauche(temp)
temp = tronque_à_droite(temp)

In [102]:
len(_)

10

In [218]:
class Logement:
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, prix: int, 
                 est_nouveau: bool, est_superhôte: bool):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_superhôte = est_superhôte
        self.est_professionnel = est_professionnel
    
    @classmethod
    def from_text(cls, texte):
        attrs = (
            'type',
            'ville',
            'description',
            'nb_lits',
            'nb_chambres',
            'est_professionnel',
            'prix',
            'est_nouveau',
            'est_superhôte'
        )
        kwargs = dict()
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        return texte.split('\n')[0].split(' ⋅ ')[0]
    
    @staticmethod
    def _get_ville(texte: str):
        return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str):
        pat = r'(?P<nb_lits>\d) lits?.+(?P<nb_chambres>\d) chambre'
        nb_lits, nb_chambres = re.search(pat, texte.split('\n')[2]).groups()
        return int(nb_lits), int(nb_chambres)
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_superhôte(texte: str):
        for ligne in texte.split('\n'):
            if ligne == 'Superhôte':
                return True
            return False
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep

In [219]:
splits = [match.start() for match in re.finditer('(Appartement|Hébergement|Tiny house) ⋅ ', temp)]
splits += [len(temp)]
splits

[0,
 222,
 436,
 682,
 923,
 1125,
 1321,
 1524,
 1786,
 2015,
 2195,
 2432,
 2673,
 2936,
 3155,
 3364,
 3617,
 3865,
 4081]

In [220]:
logements = []
for texte in [temp[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [221]:
logements

[Logement(
     type=Appartement,
     ville=Cachan,
     prix=278
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=315
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=316
 ),
 Logement(
     type=Hébergement,
     ville=Cachan,
     prix=280
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=1190
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=180
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=269
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=445
 ),
 Logement(
     type=Appartement,
     ville=Bagneux,
     prix=813
 ),
 Logement(
     type=Appartement,
     ville=L'Haÿ-les-Roses,
     prix=234
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=389
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
     prix=473
 ),
 Logement(
     type=Appartement,
     ville=L'Haÿ-les-Roses,
     prix=369
 ),
 Logement(
     type=Appartement,
     ville=Cachan,
    

In [226]:
logements[-1].nb_lits

3